In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import average_precision_score
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


#Initialize stop words for Bahasa Indonesia
id_stop_words_file = pd.read_csv('id-stopwords.txt', header=None, names=['stopwords'])
id_stop_words_list = []
for i in range(len(id_stop_words_file)):
    id_stop_words_list.append(id_stop_words_file.values[i][0])

#Initialize Dataset
dataset = pd.read_csv('data-judul-saja.csv', encoding='latin', header=0, sep=',')
input = dataset['judul_proposal'].str.lower().str.replace('[^a-zA-Z0-9 ]', '')
target = dataset['deskripsi'].str.lower()

#See dataset distribution
print(dataset.deskripsi.value_counts())


#Initialize Stemmer, Stemmernya cukup lama jadi comment aja biar cepet
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()
# for i in range(len(input)):
#    print(i)
#    input[i] = stemmer.stem(input[i])

#Initialize TFIDF Vectorizer
tvect = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)
print(tvect)

#Split Test dan Data Train
x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=4)

print("start")
print(y_test.value_counts())
print("end")

tvect1 = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)

x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)

#Oversampling data use SMOTE
sm = SMOTE(random_state=4, ratio=1)
# sm = ADASYN()
x_train_res, y_train_res = sm.fit_sample(x_traincv, y_train)

model = RandomForestClassifier(n_estimators=50, random_state=5) #around 54%
# model = LinearSVC() #around 67%
# model = LinearSVC(loss='hinge', multi_class='ovr') # 51%

# y_train = y_train_res.astype('str')
# model.fit(x_traincv,y_train)

y_train_res = y_train_res.astype('str')
model.fit(x_train_res,y_train_res)

print("Cross Validation")
predictions=model.predict(x_testcv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_test.values[i]:
     count = count + 1
print(count / len(y_test))

#Evaluation Metrics
print(accuracy_score(y_test, predictions))
print(precision_score(y_test, predictions, average='micro'))
print(recall_score(y_test, predictions, average=None))  
print(confusion_matrix(y_test, predictions))
print(f1_score(y_test, predictions, average=None))
print(f1_score(y_test, predictions, average='micro'))

print("Same Data")
predictions=model.predict(x_traincv)
count = 0
for i in range (len(predictions)):
  # print(predictions[i])
  # print(y_train.values[i])
  if predictions[i]==y_train.values[i]:
     count = count + 1
print(count / len(y_train))




#SAVE MODEL
# save the model to disk
filename = 'KlasifikasiPakar.sav'
pickle.dump(model, open(filename, 'wb'))

information management           135
intelligent system               129
software engineering             128
sistem komputer                  123
enterprise computing             118
social and professional issue     52
graphics and visualization        42
hardware                          41
algoritma dan pemrograman         22
dasar matematika                   8
Name: deskripsi, dtype: int64
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan',...'wah', 

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


Cross Validation
0.76875
0.76875
0.76875
[1.         0.80645161 0.5        0.625      0.92307692 0.88
 0.55       0.8        0.74193548]
[[ 1  0  0  0  0  0  0  0  0]
 [ 0 25  0  0  1  2  0  1  2]
 [ 0  0  4  0  1  2  0  0  1]
 [ 0  0  0  5  0  3  0  0  0]
 [ 0  0  0  0 24  0  0  0  2]
 [ 0  1  0  0  1 22  1  0  0]
 [ 0  0  1  1  3  4 11  0  0]
 [ 0  1  0  0  1  0  0  8  0]
 [ 0  4  0  1  0  0  0  3 23]]
[1.         0.80645161 0.61538462 0.66666667 0.84210526 0.75862069
 0.6875     0.72727273 0.77966102]
0.76875
Same Data
0.9639498432601881


In [2]:
from __future__ import division
import pickle
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import average_precision_score
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


#Initialize stop words for Bahasa Indonesia
id_stop_words_file = pd.read_csv('id-stopwords.txt', header=None, names=['stopwords'])
id_stop_words_list = []
for i in range(len(id_stop_words_file)):
    id_stop_words_list.append(id_stop_words_file.values[i][0])

#Initialize Dataset
dataset = pd.read_csv('data-judul-saja-silabus.csv', encoding='latin', header=0, sep=',')
input = dataset['judul_proposal'].str.lower().str.replace('[^a-zA-Z0-9 ]', '')
target = dataset['deskripsi'].str.lower()

#Initialize Stemmer, Stemmernya cukup lama jadi comment aja biar cepet
#factory = StemmerFactory()
#stemmer = factory.create_stemmer()
#for i in range(len(input)):
#    print i
#    input[i] = stemmer.stem(input[i])

#Initialize TFIDF Vectorizer
tvect = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)

#Split Test dan Data Train
x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=4)

tvect1 = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)


filename = 'KlasifikasiPakar.sav'
loaded_model = pickle.load(open(filename, 'rb'))

y_train = y_train.astype('str')
loaded_model.fit(x_traincv,y_train)

print("Cross Validation")
predictions=loaded_model.predict(x_testcv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_test.values[i]:
     count = count + 1
print(count / len(y_test))


print("Same Data")
predictions=loaded_model.predict(x_traincv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_train.values[i]:
     count = count + 1
print(count / len(y_train))

# Contoh 1 data
# x = ['aplikasi cloud di kota jakarta']
x = ['Kuliah ini membahas berbagai teknik pembelajaran oleh mesin serta pengenalan pola. Pendekatan yang diberikan mencakup supervised learning, unsupervised learning, dan reinforcement learning.']
x_new=tvect1.transform(x)
new_pred = loaded_model.predict(x_new)
print(new_pred)

filename = 'tvect.sav'
pickle.dump(tvect1, open(filename, 'wb'))

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Cross Validation
0.7359550561797753
Same Data
0.9647887323943662
['intelligent system']
